<a href="https://colab.research.google.com/github/geanfrancopalomino/repositorioprueba/blob/main/An%C3%A1lisis_sentimientos_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 0.- Importar datos de GitHub

In [7]:
! git clone https://github.com/geanfrancopalomino/repositorioprueba.git

fatal: destination path 'repositorioprueba' already exists and is not an empty directory.


#### 1.- Librerías utilizadas

In [8]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('vader_lexicon')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import string
import re
import nltk
stop_words_sp = set(stopwords.words('spanish'))

#### 2.- Carga de datos

In [10]:
df = pd.read_csv('/content/repositorioprueba/Base_Reviews.csv',sep = ";")

In [11]:
df.head(3)

,Id,Review_text
0,1,Lindo vestidoBuena telaTalla perfecta
1,2,Estoy complacido con el producto
2,3,La tela de buena calidad y entalla bien tal co...


#### 3.- Limpieza de Datos

In [12]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1929 entries, 0 to 1928
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Id           1929 non-null   int64 
 1   Review_text  1929 non-null   object
dtypes: int64(1), object(1)
memory usage: 30.3+ KB


In [14]:
def remove_punct(text):
    text = "".join([char for char in text if char not in string.punctuation])
    return text

In [15]:
df['Review_text'] = df['Review_text'].apply(lambda x: remove_punct(x))
df.head(20)

,Id,Review_text
0,1,Lindo vestidoBuena telaTalla perfecta
1,2,Estoy complacido con el producto
2,3,La tela de buena calidad y entalla bien tal co...
3,4,El productos llegó a tienda antes de lo planea...
4,5,Acabo de recoger mi pedido y me gusta mucho el...
5,6,Me quedo bien y resalta mi figura
6,7,Compre el producto pero la atencion al cliente...
7,8,PEDI UNA ZAPATILLAS 415 Y ME TRAEN UNA TALLA D...
8,9,ME CANCELARON MI PEDIDO MALASO
9,10,Recién lo compré tengo que esperar para dar un...


In [16]:
# Aplicaremos varios rounds de limpieza
import re
import string

def clean_text_round1(text):
    '''Hacer texto en minúsculas, eliminar texto entre corchetes, eliminar puntuación y eliminar palabras que contengan números.'''
    text = text.lower()
    text = re.sub('\[.*?¿\]\%', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [17]:
# vemos la primer limpieza
df['Review_text'] = pd.DataFrame(df['Review_text'].apply(round1))
df

,Id,Review_text
0,1,lindo vestidobuena telatalla perfecta
1,2,estoy complacido con el producto
2,3,la tela de buena calidad y entalla bien tal co...
3,4,el productos llegó a tienda antes de lo planea...
4,5,acabo de recoger mi pedido y me gusta mucho el...
...,...,...
1924,1925,no aguanto ni un día de uso toda la malla se d...
1925,1926,estas zapatillas son una estafa al primer dia ...
1926,1927,me encanta porque se le abre las puertas y sob...
1927,1928,compre para los cuartos de mis hijos y la cal...


In [18]:
# Segundo round
def clean_text_round2(text):
    '''Deshágase de la puntuación adicional y el texto no sensible que se perdió la primera vez.'''
    text = re.sub('[‘’“”…«»]', '', text)
    text = re.sub('\n', ' ', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [19]:
# veamos como queda
df['Review_text'] = pd.DataFrame(df['Review_text'].apply(round2))
df.head(2)

,Id,Review_text
0,1,lindo vestidobuena telatalla perfecta
1,2,estoy complacido con el producto


#### 4.- Análisis de Sentimientos

In [20]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm

sia = SentimentIntensityAnalyzer()

In [21]:
# Run the polarity score on the entire dataset
res = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
    text = row['Review_text']
    myid = row['Review_text']
    res[myid] = sia.polarity_scores(text)

  0%|          | 0/1929 [00:00<?, ?it/s]

In [22]:
#vaders = pd.DataFrame(res).T
#vaders = vaders.reset_index().rename(columns={'index': 'Id'})
#vaders = vaders.merge(df, how='left')

In [23]:
vaders = pd.DataFrame(res).T
vaders = vaders.reset_index().rename(columns={'index': 'Id'})

In [24]:
vaders.head(20)

,Id,neg,neu,pos,compound
0,lindo vestidobuena telatalla perfecta,0.000,0.556,0.444,0.3400
1,estoy complacido con el producto,0.000,1.000,0.000,0.0000
2,la tela de buena calidad y entalla bien tal co...,0.000,1.000,0.000,0.0000
3,el productos llegó a tienda antes de lo planea...,0.000,0.892,0.108,0.3182
4,acabo de recoger mi pedido y me gusta mucho el...,0.000,1.000,0.000,0.0000
5,me quedo bien y resalta mi figura,0.000,1.000,0.000,0.0000
6,compre el producto pero la atencion al cliente...,0.146,0.854,0.000,-0.5574
7,pedi una zapatillas y me traen una talla de ...,0.104,0.896,0.000,-0.2960
8,me cancelaron mi pedido malaso,0.000,1.000,0.000,0.0000
9,recién lo compré tengo que esperar para dar un...,0.000,1.000,0.000,0.0000


In [25]:
vaders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1898 entries, 0 to 1897
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Id        1898 non-null   object 
 1   neg       1898 non-null   float64
 2   neu       1898 non-null   float64
 3   pos       1898 non-null   float64
 4   compound  1898 non-null   float64
dtypes: float64(4), object(1)
memory usage: 74.3+ KB


In [26]:
vaders.to_csv('archivo.csv', index=False)